In [1]:
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from sklearn import metrics
import matplotlib.pyplot as plt
from IPython.display import IFrame
from sklearn.cluster import KMeans
import matplotlib.font_manager as fm
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Circle, Arc

#plt.style.use('ggplot')
%matplotlib inline
pd.options.display.max_columns=25

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
import mechanize

In [3]:
import cookielib
import json

cap_url = "http://stats.nba.com/stats/commonallplayers?"
cap_param = {'IsOnlyCurrentSeason':"1",
                      'LeagueID': u'00',
                      'Season': u'2015-16'}
cap_resp = requests.get(cap_url, params=cap_param)
if cap_resp.status_code != 200:
    br = mechanize.Browser()
    cj = cookielib.LWPCookieJar()
    br.set_cookiejar(cj)
    br.set_handle_equiv(True)
    br.set_handle_gzip(True)
    br.set_handle_redirect(True)
    br.set_handle_referer(True)
    br.set_handle_robots(False)
    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    data = br.open(cap_resp.url).get_data()
    jdata = json.loads(data)
else:
    jdata = cap_resp.json()
cap_df = pd.DataFrame(jdata['resultSets'][0]['rowSet'],columns=jdata['resultSets'][0]['headers'])
cap_df.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: UserWarning: gzip transfer encoding is experimental!


,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
0,203112,"Acy, Quincy",1,2012,2015,quincy_acy,1610612758,Sacramento,Kings,SAC,kings,Y
1,203919,"Adams, Jordan",1,2014,2015,jordan_adams,1610612763,Memphis,Grizzlies,MEM,grizzlies,Y
2,203500,"Adams, Steven",1,2013,2015,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
3,201167,"Afflalo, Arron",1,2007,2015,arron_afflalo,1610612752,New York,Knicks,NYK,knicks,Y
4,201582,"Ajinca, Alexis",1,2008,2015,alexis_ajinca,1610612740,New Orleans,Pelicans,NOP,pelicans,Y


In [4]:
def format_name_last_comma_first(name):
    if len(name.split())<2:
        return 'All'
    if ',' in name:
        names = name.split(',')
    else:
        names = name.split()
    return names[-1].strip()+', '+' '.join(names[0:-1]).strip()
print(format_name_last_comma_first('James, Lebron'))

Lebron, James


In [5]:
def player_lookup(name='All',season='2015-16'):
    cap_url = "http://stats.nba.com/stats/commonallplayers?"
    cap_param = {'IsOnlyCurrentSeason':"1",
                      'LeagueID': u'00',
                      'Season': season}
    cap_resp = requests.get(cap_url, params=cap_param)
    cap_df = pd.DataFrame(cap_resp.json()['resultSets'][0]['rowSet'],columns=cap_resp.json()['resultSets'][0]['headers'])
    return cap_df

In [6]:
pname = 'Blake Griffin'
cap_df = player_lookup()
pinfo = cap_df[cap_df['DISPLAY_LAST_COMMA_FIRST']==format_name_last_comma_first(pname)]
pID = pinfo['PERSON_ID'].values
print(pinfo.head())
print(pID)

     PERSON_ID DISPLAY_LAST_COMMA_FIRST  ROSTERSTATUS FROM_YEAR TO_YEAR  \
169     201933           Griffin, Blake             1      2009    2015   

        PLAYERCODE     TEAM_ID    TEAM_CITY TEAM_NAME TEAM_ABBREVIATION  \
169  blake_griffin  1610612746  Los Angeles  Clippers               LAC   

    TEAM_CODE GAMES_PLAYED_FLAG  
169  clippers                 Y  
[201933]


In [8]:
sc_params = {u'AheadBehind': u'',
 u'ClutchTime': u'',
 u'ContextFilter': u'',
 u'ContextMeasure': u'FGA',
 u'DateFrom': u'',
 u'DateTo': u'',
 u'EndPeriod': u'',
 u'EndRange': u'',
 u'GameID': u'',
 u'GameSegment': u'',
 u'LastNGames': 0,
 u'LeagueID': u'00',
 u'Location': u'',
 u'Month': 0,
 u'OpponentTeamID': 0,
 u'Outcome': u'',
 u'Period': 0,
 u'PlayerID': pID,
 u'PointDiff': u'',
 u'Position': u'',
 u'RangeType': u'',
 u'RookieYear': u'',
 u'Season': u'2015-16',
 u'SeasonSegment': u'',
 u'SeasonType': u'Regular Season',
 u'StartPeriod': u'',
 u'StartRange': u'',
 u'TeamID': 0,
 u'VsConference': u'',
 u'VsDivision': u''}

sc_url = 'http://stats.nba.com/stats/shotchartdetail?'
sc_resp = requests.get(sc_url,params=sc_params)
print(sc_resp.url) 
if sc_resp.status_code != 200:
    br = mechanize.Browser()
    cj = cookielib.LWPCookieJar()
    br.set_cookiejar(cj)
    br.set_handle_equiv(True)
    br.set_handle_gzip(True)
    br.set_handle_redirect(True)
    br.set_handle_referer(True)
    br.set_handle_robots(False)
    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    data = br.open(sc_resp.url).get_data()
    jdata = json.loads(data)
else:
    jdata = sc_resp.json()
player_shot_df = pd.DataFrame(jdata['resultSets'][0]['rowSet'],columns=jdata['resultSets'][0]['headers'])
print(player_shot_df.shape)
player_shot_df.head(5)

http://stats.nba.com/stats/shotchartdetail?PlayerID=201933&StartPeriod=&StartRange=&TeamID=0&VsConference=&Location=&SeasonType=Regular+Season&RangeType=&RookieYear=&Season=2015-16&DateFrom=&ClutchTime=&EndPeriod=&EndRange=&ContextFilter=&GameID=&OpponentTeamID=0&DateTo=&GameSegment=&AheadBehind=&PointDiff=&LastNGames=0&VsDivision=&LeagueID=00&Position=&Outcome=&ContextMeasure=FGA&SeasonSegment=&Period=0&Month=0
(553, 21)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: gzip transfer encoding is experimental!


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,Shot Chart Detail,0021500016,4,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,11,1,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,22,1,228,1,1
1,Shot Chart Detail,0021500016,8,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,10,28,Made Shot,Cutting Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,-32,26,1,1
2,Shot Chart Detail,0021500016,30,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,8,9,Made Shot,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-155,134,1,1
3,Shot Chart Detail,0021500016,50,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,6,55,Made Shot,Driving Finger Roll Layup Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,0,43,1,1
4,Shot Chart Detail,0021500016,52,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,6,25,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,21,82,200,1,0


In [9]:
#Get all missed shots from mid-range, 16-24ft.
miss_mid_df = player_shot_df[(player_shot_df['EVENT_TYPE']=='Missed Shot') &
                             (player_shot_df['SHOT_ZONE_BASIC']=='Mid-Range')]
miss_mid_df.head(10)

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
4,Shot Chart Detail,0021500016,52,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,6,25,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Right Side Center(RC),16-24 ft.,21,82,200,1,0
14,Shot Chart Detail,0021500016,344,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,7,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,21,-50,208,1,0
16,Shot Chart Detail,0021500016,420,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,2,55,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,17,10,175,1,0
18,Shot Chart Detail,0021500016,503,201933,Blake Griffin,1610612746,Los Angeles Clippers,4,7,32,Missed Shot,Fadeaway Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,14,-148,11,1,0
19,Shot Chart Detail,0021500016,511,201933,Blake Griffin,1610612746,Los Angeles Clippers,4,5,56,Missed Shot,Turnaround Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-178,70,1,0
20,Shot Chart Detail,0021500020,7,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,11,9,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-91,159,1,0
23,Shot Chart Detail,0021500020,83,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,4,40,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,16,-4,169,1,0
24,Shot Chart Detail,0021500020,179,201933,Blake Griffin,1610612746,Los Angeles Clippers,2,8,58,Missed Shot,Jump Bank Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-145,124,1,0
29,Shot Chart Detail,0021500020,249,201933,Blake Griffin,1610612746,Los Angeles Clippers,2,1,43,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,21,-78,198,1,0
31,Shot Chart Detail,0021500020,320,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,10,39,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,19,-2,198,1,0


In [10]:
miss_mid_df['SHOT_ZONE_AREA'].value_counts()

Center(C)                57
Left Side Center(LC)     51
Left Side(L)             32
Right Side Center(RC)    24
Right Side(R)            11
Name: SHOT_ZONE_AREA, dtype: int64

In [11]:
cnt_miss_mid_df=miss_mid_df[miss_mid_df['SHOT_ZONE_AREA']=='Center(C)']
cnt_miss_mid_df.head(6)

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
14,Shot Chart Detail,0021500016,344,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,7,21,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,21,-50,208,1,0
16,Shot Chart Detail,0021500016,420,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,2,55,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,17,10,175,1,0
23,Shot Chart Detail,0021500020,83,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,4,40,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,16,-4,169,1,0
31,Shot Chart Detail,0021500020,320,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,10,39,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,19,-2,198,1,0
39,Shot Chart Detail,0021500038,45,201933,Blake Griffin,1610612746,Los Angeles Clippers,1,7,27,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,18,-50,180,1,0
45,Shot Chart Detail,0021500038,399,201933,Blake Griffin,1610612746,Los Angeles Clippers,3,8,8,Missed Shot,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,18,-37,183,1,0


**On January 25th, 2016, the NBA stats website announced this news:
'We are currently experiencing technical difficulties with player tracking movement and player tracking logs We are working to solve the problem and apologize for any inconvenience this may cause.'**

In [14]:
for index, row in cnt_miss_mid_df.iterrows():
    url = 'http://stats.nba.com/stats/locations_getmoments/?eventid='+ str(row['GAME_EVENT_ID']) + \
            '&gameid=' + str(row['GAME_ID'])
    print(url)
    response = requests.get(url)
    if response.status_code != 200:
        br = mechanize.Browser()
        cj = cookielib.LWPCookieJar()
        br.set_cookiejar(cj)
        br.set_handle_equiv(True)
        br.set_handle_gzip(True)
        br.set_handle_redirect(True)
        br.set_handle_referer(True)
        br.set_handle_robots(False)
        br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
        br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
        data = br.open(response.url).get_data()
        jdata = json.loads(data)
    else:
        jdata = response.json()
    moments = jdata["moments"]
    moment_data = []
    for m in moments:
        for p in m[5]:
            p+=[moments.index(m), m[2],m[3]]
            moment_data.append(p)
    event_track_dict
pm_df = pd.DataFrame(data=moment_data,columns=['teamID','playerID','xloc','yloc','zloc','moment','gametime','shottime'])
pm_df.head(15)

http://stats.nba.com/stats/locations_getmoments/?eventid=344&gameid=0021500016


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: UserWarning: gzip transfer encoding is experimental!


httperror_seek_wrapper: HTTP Error 404: Not Found

In [ ]:
def get_moments(gameid=None,eventid=None):
    if gameid is None or eventid is None:
        ValueError('Must supply a gameid and a eventid')
    url = 'http://stats.nba.com/stats/locations_getmoments/?eventid='+ str(eventid) + \
            '&gameid=' + str(gameid)
    print(url)
    response = requests.get(url)
    moments = response.json()["moments"]
    moment_data = []
    for m in moments:
        for p in m[5]:
            p+=[moments.index(m), m[2],m[3]]
            moment_data.append(p)
    pm_df = pd.DataFrame(data=moment_data,columns=['teamID','playerID','xloc','yloc','zloc','moment','gametime','shottime'])
    return pm_df
